# Movie Recommendation System: Model-Based Collaborative Filtering Model (Matrix Factorization)

**Background**:

An accurate, personalized recommendation system can improve business and sales and build customer satisfaction.


**Types of Recommendation Engines**:
1. **Popularity Model (see "popularity_movie_recommendation" notebook)**
2. **Recommendation algorithms**
    - **Content-based filtering (see "content_based_movie_recommendation" notebook)**
    - **Memory-based collaborative filtering (see "memory_based_collaborative_filtering_movie_recommendation" notebook)**
    - **Model-based collaborative filtering**
        - **Uses parametric machine learning to find user ratings of unrated items**
        - Examples: PCA, SVD, Neural Nets, Matrix Factorization
        - Learns parameters using gradient descent or other optimization algorithms
        - Applies dimensionality reduction techniques to capture more signals from the big datasets
        - **Advantages**:
            - Dimensionality reduction handles sparse/missing data
            - Avoids computational issues with large numbers of predictors
            - Easier to visualize the data
            - Easier to data mine with fewer predictors
            - Uncover hidden correlations/features in the raw data
            - Remove redundant and noise features that are useless
            - Access easier data storage and processing
        - **Disadvantages**:
            - Inference is intracable because factors are hidden/latent
        - Types of model-based collaborative filtering algorithms:
            1. **Matrix Factorization Based**
                - Unsupervised 
                - Deals better with scalability and sparsity than memory-based CF
                - Learns the latent prefernces of users and latent attributse of items from known ratings to predict the unknown ratings through the dot product of the latent features of users and items
                - Restructures the user_item matrix into low-rank structure and represent the matrix by the multiplication of two low-rank matrices where the rows contain the latent vector
                - Multiply the low-rank matrices together to approximate the original matrix and fill in any missing entries
                1. **Singular Vector Decomposition (see "model_based_cf_matrix_factorization_movie_recommendation" notebook)**
                    - Algorithm that decomposes a matrix A into two unitary matrices and a diagonal matrix
                    - A: input data matrix (ie user's ratings)
                    - U: left singular vectors (user "features" matrix) that represents how much users like each feature
                    - D: diagonal matrix of singular values (weights/strengths of each concept)
                    - V^T: right singular vectors (movie "features" matrix) that represents how relevant each feature is to each movie
                    - Reduces the dimension of the dataset
                    - Gives low-rank approximation of user tastes and preferences
                    - Need to extract user preferences that can be determined by a small number of hidden factors called **embeddings**
                    - **Embeddings** are low dimensional hidden factors for items and users
                    - Python packages for implementation: surprise
                2. **Probabilitistic Matrix Factorization**
                    - Python packages for implementation: fastai
                3. **Non -ve Matrix Factorization**
                    - Python packages for implementation: surprise
            2. **Deep learning/neural network (see "model_based_cf_deep_learning_movie_recommendation_model" notebook)**
                
3. **Using a classifier to make recommendation**
    - Classifiers are parametric solutions that require some parameters of the user and item to be defined first
    - Pros:
        - Incorporates personalization
        - Works even if the user's past history is short or not available
    - Cons:
        - Features might not be availalbe or sufficient to create a good classifier
        - Making a good classifier will become exponentially difficult as the number of user and items grow
        
(https://medium.com/@james_aka_yale/the-4-recommendation-engines-that-can-predict-your-movie-tastes-bbec857b8223)

**Data**:

We will be using the online movie recommender service MovieLens' dataset collected from the MovieLens website. The datasets were collected over several periods of time.
Users were selected at random to be included in the data. All users have rated 20+ movies. No demographic information is included.

The data includes:
- 100K ratings (1-5) from 1000 users on 1700 movies
- Each user has rated 20+ movies
- Simple demographic information for the users, such as gender, age, occupation, zip, etc.
- Genre information of movies

(https://grouplens.org/datasets/movielens/10m/)

In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Data

## Users Data

In [2]:
users = pd.read_pickle('users.pickle')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


## Ratings Data

In [3]:
ratings = pd.read_pickle('ratings.pickle')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


## Movies Data

In [4]:
movies = pd.read_pickle('movies.pickle')
movies.drop(movies.columns[2:24], axis=1, inplace=True)
movies.head()

,movie_id,movie_title,genres
0,1,Toy Story (1995),animation|childrens|comedy
1,2,GoldenEye (1995),action|adventure|thriller
2,3,Four Rooms (1995),thriller
3,4,Get Shorty (1995),action|comedy|drama
4,5,Copycat (1995),crime|drama|thriller


# Model-Based Collaborative Filtering Movie Recommendation Model: Matrix Factorization with SVD

(https://medium.com/@james_aka_yale/the-4-recommendation-engines-that-can-predict-your-movie-tastes-bbec857b8223)

**Let's count the number of unique users and movies.**

In [5]:
num_users = ratings['user_id'].unique().shape[0]
num_movies = ratings['movie_id'].unique().shape[0]

print 'Number of users:', num_users
print 'Number of movies:', num_movies

Number of users: 943
Number of movies: 1682


**We need to format the ratings matrix to be one row per user and one column per movie.**

In [6]:
ratings_pivot = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
ratings_pivot.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,...,1433,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454,1455,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,2.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,3.0,5.0,4.0,5.0,4.0,5.0,5.0,4.0,3.0,2.0,5.0,4.0,4.0,3.0,4.0,3.0,3.0,3.0,4.0,3.0,1.0,4.0,4.0,4.0,1.0,4.0,4.0,5.0,5.0,3.0,4.0,3.0,5.0,5.0,4.0,5.0,4.0,5.0,3.0,5.0,2.0,4.0,5.0,3.0,4.0,3.0,5.0,2.0,2.0,1.0,1.0,2.0,4.0,4.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,5.0,3.0,3.0,3.0,5.0,1.0,4.0,3.0,4.0,5.0,3.0,2.0,5.0,4.0,5.0,3.0,1.0,4.0,4.0,4.0,4.0,3.0,5.0,1.0,3.0,1.0,3.0,2.0,1.0,4.0,2.0,4.0,3.0,2.0,2.0,5.0,4.0,5.0,3.0,5.0,2.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0,5.0,4.0,5.0,4.0,4.0,4.0,4.0,3.0,3.0,5.0,5.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,4.0,3.0,3.0,5.0,4.0,5.0,3.0,4.0,5.0,5.0,4.0,4.0,3.0,4.0,2.0,4.0,3.0,5.0,3.0,3.0,1.0,3.0,5.0,4.0,5.0,5.0,2.0,3.0,4.0,5.0,4.0,4.0,1.0,3.0,2.0,4.0,5.0,4.0,2.0,4.0,4.0,3.0,4.0,5.0,1.0,2.0,2.0,5.0,1.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

**Next, we need to denormalize the data (normalize by each user's mean) and convert it from a dataframe to a numpy array.**

In [7]:
ratings_pivot_matrix = ratings_pivot.as_matrix()
user_ratings_mean = np.mean(ratings_pivot_matrix, axis=1)
ratings_normalized = ratings_pivot_matrix - user_ratings_mean.reshape(-1, 1)

**Let's check the sparsity of the ratings dataset. When there is a very sparse matrix with lots of dimensions, doing matrix factorization can restructure the user-item matrix into low-rank structure to better approximate the original matrix.**

In [8]:
sparsity = round(1 - len(ratings)/float(num_users * num_movies), 3)
print 'The sparsity level of MovieLens100K dataset is', sparsity * 100, '%.'

The sparsity level of MovieLens100K dataset is 93.7 %.


**We will use the Scipy function `svds` because it allows us to choose how many latent factors to use to approximate the original ratings matrix.**

In [9]:
from scipy.sparse.linalg import svds
U, diags, Vt = svds(ratings_normalized, k=50)

In [10]:
D = np.diag(diags)

**We can start making movie predictions for every user from the decomposed matrices. But first we need to add the user means back to get the actual star ratings prediction.**

In [11]:
all_user_pred_ratings = np.dot(np.dot(U, D), Vt) + user_ratings_mean.reshape(-1, 1)

In [12]:
preds = pd.DataFrame(all_user_pred_ratings, columns=ratings_pivot.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,...,1433,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454,1455,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,6.488436,2.959503,1.634987,3.024467,1.656526,1.659506,3.630469,0.240669,1.791518,3.347816,2.966128,5.873074,4.101069,3.389306,4.106865,2.365477,2.051866,0.874133,3.430062,2.858839,0.867343,2.910597,2.212223,3.262437,4.399117,0.821052,1.236004,3.681455,0.583801,1.966995,3.699791,2.086317,2.275488,0.107836,0.302267,0.098060,0.390855,1.287024,2.429299,0.923861,0.293912,3.334737,1.046127,2.125516,3.148509,0.847907,3.611339,1.611321,2.307993,5.608465,0.741138,1.144871,1.456105,2.223851,3.684991,4.517366,1.174318,2.168867,4.942899,4.039338,3.309326,1.863975,1.118751,4.534535,3.181996,1.198788,1.866837,3.071543,4.892945,3.936790,2.273450,1.739154,2.207839,0.305892,0.726822,0.557505,2.177733,0.146881,2.264251,2.152591,4.174990,3.634308,2.189742,0.936213,1.452871,1.646372,4.807039,3.133743,6.711143,1.588641,2.951726,1.600362,1.551082,1.535893,3.502176,4.879115,2.973201,5.344757,0.248126,7.558040,1.801997,0.494464,0.190717,0.115783,1.249732,1.193688,0.971799,2.079902,4.025532,0.157100,2.819177,0.249827,1.505223,2.912911,1.242792,2.564737,1.585004,0.699503,0.987780,0.732848,1.633770,1.679707,1.408618,3.589457,2.542732,1.794945,4.773624,2.372239,2.484785,0.739868,0.584046,2.765883,1.781313,3.065290,5.680061,2.183779,3.520059,0.307302,0.858638,0.274902,1.429935,0.262193,0.167982,4.077501,1.315970,0.868073,2.004982,1.062698,0.861949,4.348056,3.557386,1.589309,4.534839,4.270741,1.334718,4.786115,4.026566,1.879015,2.558480,2.279085,2.745865,2.382294,2.766001,2.752556,2.652652,2.935661,1.639770,5.295662,3.712385,2.920086,1.836562,4.042819,3.920170,4.441769,4.654532,6.683531,2.957780,2.265215,4.437784,4.341606,3.727169,3.689881,5.663938,3.025496,2.084134,3.910041,2.552406,3.435352,2.374156,3.146244,4.629102,0.759868,0.753244,2.139217,4.009049,5.022452,4.146370,4.959577,2.516724,4.332671,2.642420,4.408699,2.668161,2.967262,1.31538

**We now create a function taht returns the movies with the highest predicted ratings that the specified user hasn't rated already.**

In [13]:
def rec_movies(predictions, user_id, movies, original_ratings, num_rec):
    
    # Get and sort the user's predictions
    user_row_num = user_id - 1
    sorted_user_preds = preds.iloc[user_row_num].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie info
    user_data = original_ratings[original_ratings['user_id']==user_id]
    user_full = user_data.merge(movies, how='left', on='movie_id').sort_values(['rating'], ascending=False)
    print 'User {0} has already rated {1} movies.'.format(user_id, user_full.shape[0])
    print 'Recommending highest {0} predicted rating movies not already rated.'.format(num_rec)
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet
    recs = movies[~movies['movie_id'].isin(user_full['movie_id'])].merge(pd.DataFrame(sorted_user_preds).reset_index(), how='left', on='movie_id').rename(columns={user_row_num: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_rec, :-1]
    
    return user_full, recs

**Let's recommend 20 movies to user id 210.**

In [14]:
already_rated, predictions = rec_movies(preds, 210, movies, ratings, 20)

User 210 has already rated 132 movies.
Recommending highest 20 predicted rating movies not already rated.


**Here are the top 20 movies user 210 has rated.**

In [15]:
already_rated.head(20)

,user_id,movie_id,rating,timestamp,movie_title,genres
53,210,152,5,887730676,Sleeper (1973),comedy|sci_fi
44,210,182,5,887736232,GoodFellas (1990),crime|drama
29,210,73,5,891035837,Maverick (1994),action|comedy|western
82,210,483,5,887736482,Casablanca (1942),drama|romance|war
34,210,50,5,887731014,Star Wars (1977),action|adventure|romance|sci_fi|war
81,210,181,5,887731082,Return of the Jedi (1983),action|adventure|romance|sci_fi|war
125,210,168,5,887730342,Monty Python and the Holy Grail (1974),comedy
79,210,482,5,887736739,Some Like It Hot (1959),comedy|crime
124,210,302,5,890059415,L.A. Confidential (1997),crime|film_noir|mystery|thriller
41,210,257,5,887730789,Men in Black (1997),action|adventure|comedy|sci_fi


**Here are the top 20 movies that our model-based CF model using matrix factorization recommends.**

In [16]:
predictions

,movie_id,movie_title,genres
135,183,Alien (1979),action|horror|sci_fi|thriller
142,194,"Sting, The (1973)",comedy|crime
63,82,Jurassic Park (1993),action|adventure|sci_fi
111,143,"Sound of Music, The (1965)",musical
112,144,Die Hard (1988),action|thriller
103,133,Gone with the Wind (1939),drama|romance|war
404,504,Bonnie and Clyde (1967),crime|drama
52,66,While You Were Sleeping (1995),comedy|romance
171,239,Sneakers (1992),crime|drama|sci_fi
91,118,Twister (1996),action|adventure|thriller


**Although we didn't actually use the genre of the movies as a feature, the truncated matrix factorization features "picked up" on the underlying tastes of the user, as seen by the genres of the recommended movies and those of the user's top rated movies.**

# Evaluation the Model-Based CF Model Using Matrix Factorization

**We will use the `suprise` library instead because it provides powerful ready-to-use prediction algorithms including SVD to evaluate its RMSE. The `surprise` library is a Python scikit building and analyzing recommender systems.**

In [20]:
from surprise import Reader, Dataset, SVD, evaluate

In [23]:
# Load reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Split the dataset for 5-fold evaluation
data.split(n_folds=5)

In [24]:
svd = SVD()
evaluate(svd, data, measures=['RMSE'])

Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.9375
------------
Fold 2
RMSE: 0.9369
------------
Fold 3
RMSE: 0.9323
------------
Fold 4
RMSE: 0.9363
------------
Fold 5
RMSE: 0.9351
------------
------------
Mean RMSE: 0.9356
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.93754135160943108,
                             0.93685829553426336,
                             0.93228252758368224,
                             0.93629478980283276,
                             0.93509856910237676]})

**The mean RMSE is 0.9356, which is pretty good. Let's now train on the dataset and get the predictions.**

In [25]:
trainset = data.build_full_trainset()
svd.train(trainset)

**Let's use the sample user 210 again.**

In [26]:
ratings[ratings['user_id']==210]

,user_id,movie_id,rating,timestamp
13,210,40,3,891035994
1018,210,204,5,887730676
1155,210,70,4,887730589
1354,210,97,5,887736454
1417,210,527,5,887736232
1435,210,274,5,887730676
1850,210,357,5,887736206
2081,210,161,5,887736393
2392,210,380,4,887736482
2626,210,58,4,887730177


**Let use SVD to predict the rating that user 210 will give to a random movie, ie movie id 835.**

In [27]:
svd.predict(210, 835)

Prediction(uid=210, iid=835, r_ui=None, est=4.2458539630091527, details={u'was_impossible': False})

**The estimated rating is 4.246. The recommender system works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.**

**Although the method of using matrix factorization (low-rank approximation) can capture the underlying features driving the raw data to scale better to large datasets and make better recommendations based on user tastes, we lose some meaningful signals. There is an interpretability problem as singular vector specifies a linear combination of all input columns or rows and there is a lack of sparsity when the singular vectors are quite dense; hence, the SVD method is limited to linear projections.**